In [ ]:
import pandas as pd
df_AM = pd.read_excel("Wahania_Modulacja_Nosna_Sinus.xlsx", sheet_name="AM")
df_AM.drop(columns=df_AM.filter(regex='Unnamed').columns, inplace=True)

In [ ]:
df_AM

,Uc [V],230,Lp.,fm [Hz],DUU_1 [%],PST/1,DUU_3 [%],PST/3,DUU_0333 [%],"PST/0,333"
0,fc [Hz],50,1,0.010000,2.570586,0.999787,7.732059,2.997543,0.856535,0.333324
1,Pst [-],const,2,0.010471,2.519499,0.999796,7.595150,2.997915,0.838960,0.333353
2,Iph [deg],0,3,0.010965,2.465303,0.999558,7.417241,3.002661,0.821240,0.333415
3,NaN,RECTANGLE,4,0.011482,2.511468,0.999691,7.542087,2.999935,0.837952,0.333310
4,NaN,NaN,5,0.012023,2.457252,0.999827,7.404683,2.999523,0.819590,0.333326
...,...,...,...,...,...,...,...,...,...,...
269,NaN,NaN,270,138.038400,9.168889,1.000123,27.975775,2.998728,3.043681,0.333357
270,NaN,NaN,271,141.000000,15.076475,1.000076,47.601681,2.998216,4.975392,0.333188
271,NaN,NaN,272,144.544000,27.405336,1.000613,104.569025,2.998045,8.952598,0.333110
272,NaN,NaN,273,147.000000,40.807305,0.999866,190.000000,2.447389,12.928682,0.333159


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from scipy.signal import hilbert, square, sawtooth
from matplotlib import pyplot as plt, animation
import numpy as np
import os

fs =20480.0  # czestotliwosc probkowania
T =600.0  # czas trwania
dt = 1 / fs  # krok czasowy
n = T * fs  # liczba probek sygnału
t = np.arange(n) * dt  # wektor czasu
A = 230 * np.sqrt(2)
f=50.0

def sygnal(A,f):
    return  (1 + (DUU / 200) * square(2 * np.pi * fm * t)) * (A * np.sin(2 * np.pi * f * t))

file_path='/content/drive/MyDrive/wyniki_modulacja_AM.csv'

if os.path.exists(file_path):
    df=pd.read_csv(file_path)

else:
    df=pd.DataFrame([])
nazwy_kolumn = ['AM_3_A','AM_3_f']
df2 = pd.DataFrame([], columns=nazwy_kolumn)
for n, (i,j) in enumerate(zip(df_AM["fm [Hz]"][0:4], df_AM["DUU_3 [%]"][0:4])):
  DUU=j
  fm=i
  obwiednia = hilbert(sygnal(A,f))

  amplituda = np.abs(obwiednia)
  faza = np.angle(obwiednia)
  czestotliwosc = (np.diff(faza) / (2.0*np.pi) * fs)

  b_w_A = ((abs(((1 + (DUU / 200) * square(2 * np.pi * fm * t)) * A) - amplituda)) / ((1 + (DUU / 200) * square(2 * np.pi * fm * t)) * A)) * 100
  b_w_A=b_w_A[::205]

  b_w_f= ((abs(f - czestotliwosc)) / f) * 100
  b_w_f=b_w_f[::205]

  df3 = pd.DataFrame(zip(b_w_A, b_w_f), columns=nazwy_kolumn)
  df2 = pd.concat([df2, df3], ignore_index=True)
  print(n)
  print(amplituda)
  print(czestotliwosc)
  print(b_w_f)

if df.empty:
  df=df2
else:
  df=df.merge(df2, left_index=True, right_index=True)

df.to_csv(file_path,index=False)



0
[325.26911934 325.8701384  326.23112029 ... 323.94176024 324.30750892
 324.66820402]
[51.83738418 51.72353947 51.60836801 ... 48.43158597 48.37037768
 48.26214123]
[3.67476836 0.65809834 0.36144951 ... 0.00371635 0.03708016 0.4050808 ]
1
[337.62376845 337.62376491 337.62376737 ... 337.62376275 337.62376738
 337.62376491]
[49.99965835 49.99965746 49.99965956 ... 49.99965686 49.99965962
 49.99965752]
[6.83301923e-04 6.84377753e-04 6.82715033e-04 ... 1.56717553e-05
 1.43454673e-04 1.99398013e-05]
2
[325.26893378 325.8455229  326.19175848 ... 323.99564637 324.34646864
 324.69252639]
[51.76272837 51.65367798 51.54330166 ... 48.49598055 48.43715687
 48.33321532]
[3.52545674e+00 6.32481717e-01 3.47247345e-01 ... 2.71309985e-03
 3.63008269e-02 3.87781827e-01]
3
[325.26913194 325.8553748  326.20749246 ... 323.974367   324.33114299
 324.68297121]
[51.7923213  51.68136031 51.56910556 ... 48.47041483 48.41067551
 48.30500897]
[3.5846426  0.64261275 0.35286107 ... 0.0038186  0.03589297 0.39534528